# Core Imports

In [ ]:
# Custom Imports
...

# File I/O
from pathlib import Path

# Cheminformatics
from rdkit import Chem

In [ ]:
# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

# File and chemistry type definitions

In [ ]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Building PDB files from monomer fragments

In [ ]:
from polymerist.genutils.containers import RecursiveDict
from polymerist.genutils.fileutils.pathutils import asstrpath

from polymerist.monomers import MonomerGroup
from polymerist.polymers import building


pdb_dir = Path('pdb_files')
pdb_dir.mkdir(exist_ok=True)

DOP = 10

failed = RecursiveDict()
for chemistry in chemistries:
    mono_src_dir = MONO_INFO_DIR / chemistry
    chem_dir = pdb_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mono_path in mono_src_dir.iterdir():
        monogrp = MonomerGroup.from_file(mono_path)
        try:
            polymer = building.build_linear_polymer(monomers=monogrp, DOP=DOP, sequence='AB')  
            savepath = chem_dir / f'{mono_path.stem}.pdb'
            building.mbmol_to_openmm_pdb(savepath, polymer)
            
            # polymer.save(asstrpath(savepath))
        except Exception as e:
            failed[chemistry][e.__class__.__name__][mono_path.stem] = monogrp

## Testing sanitization of monomers prior to mbuild conversion

In [ ]:
from polymerist.rdutils import rdkdraw
from polymerist.rdutils.amalgamation.bonding import hydrogenate_rdmol_ports
from polymerist.rdutils.labeling import molwise

from polymerist.monomers import specification
rdkdraw.set_rdkdraw_size(300, 3/2)


chemistry = 'urethane'
idx = 20

monogrp = MonomerGroup.from_file(MONO_INFO_DIR / chemistry / f'{chemistry}_{idx}.json')

mbmols = {}
def show(monogrp : MonomerGroup) -> None:
    for resname, mol in monogrp.iter_rdmols():
        # mol = molwise.clear_atom_map_nums(mol, in_place=False)
        # Chem.SanitizeMol(mol, sanitizeOps=specification.SANITIZE_AS_KEKULE)
        print(resname)
        display(mol)

        mbmol, linker_ids = building.mbmol_from_mono_rdmol(mol)
        mbmols[resname] = mbmol
        # mbmol, ports = building.mbmol_from_mono_smarts(new_smarts)
        # prot_mol = hydrogenate_rdmol_ports(mol, in_place=False)

show(monogrp)

In [ ]:
for resname, mol in monogrp.iter_rdmols():
    print(resname)
    display(mol)